# ニュース記事のジャンル予測（9分類問題）

事前学習済み日本語BERTモデルを、分類タスク用に転移学習（ファインチューニング）します。

学習は次の論文に従い、Linear Probingの後、Fine-tuningします。

- [Fine-Tuning can Distort Pretrained Features and Underperform Out-of-Distribution](https://arxiv.org/abs/2202.10054)

今回は入出力が次の形式を持ったタスク用に転移学習します。

- **入力**: "{title} {body}"をトークナイズしたトークンID列（最大512トークン）
- **出力**: {genre_id}

ここで、{title}はニュース記事のタイトル、{body}は本文、{genre_id}はニュースの分類ラベル（0〜8）です。


# ライブラリやデータの準備

## 依存ライブラリのインストール

In [1]:
!pip install -qU torch==1.13.* torchtext==0.14.* torchvision==0.14.* torchaudio==0.13.* torchmetrics==0.11.* \
    transformers==4.26.1 pytorch_lightning==1.9.3 fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 826.4/826.4 KB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 KB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 74.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.4 MB/s eta 0:

In [2]:
!pip list | grep -e "torch" -e "transformers" -e "pytorch_lightning" -e "fugashi" -e "ipadic"

fugashi                       1.2.1
ipadic                        1.0.0
pytorch-lightning             1.9.3
torch                         1.13.1+cu116
torchaudio                    0.13.1+cu116
torchmetrics                  0.11.4
torchsummary                  1.5.1
torchtext                     0.14.1
torchvision                   0.14.1+cu116
transformers                  4.26.1


## 各種ディレクトリ作成

* data: 学習用データセット格納用
* model: 学習済みモデル格納用（Linear Probing + Fine-tuning）
* lp_model: 学習済みモデル格納用（Linear Probing）


In [3]:
!mkdir -p /content/data /content/model /content/lp_model

In [4]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"

# Linear Probing済みモデルを保存する場所
LP_MODEL_DIR = "/content/lp_model"

# Linear Probing + Fine-tuning済みモデルを保存する場所
MODEL_DIR = "/content/model"

## livedoor ニュースコーパスのダウンロード

In [5]:
!wget -O ldcc-20140209.tar.gz https://www.rondhuit.com/download/ldcc-20140209.tar.gz

--2023-03-12 08:53:02--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.78MB/s    in 6.7s    

2023-03-12 08:53:10 (1.26 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



## livedoorニュースコーパスの形式変換

livedoorニュースコーパスを次の形式のTSVファイルに変換します。

* 1列目: タイトル
* 2列目: 本文
* 3列目: ジャンルID（0〜8）

TSVファイルは/content/dataに格納されます。


## 文字列の正規化の定義

表記揺れを減らします。今回は[neologdの正規化処理](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)を一部改変したものを利用します。
処理の詳細はリンク先を参照してください。

In [6]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

## 情報抽出

ニュース記事のタイトルと本文とジャンル（9分類）の情報を抽出します。

In [7]:
import tarfile
import re

target_genres = ["dokujo-tsushin",
                 "it-life-hack",
                 "kaden-channel",
                 "livedoor-homme",
                 "movie-enter",
                 "peachy",
                 "smax",
                 "sports-watch",
                 "topic-news"]

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body

genre_files_list = [[] for genre in target_genres]

all_data = []

with tarfile.open("ldcc-20140209.tar.gz") as archive_file:
    for archive_item in archive_file:
        for i, genre in enumerate(target_genres):
            if genre in archive_item.name and archive_item.name.endswith(".txt"):
                genre_files_list[i].append(archive_item.name)

    for i, genre_files in enumerate(genre_files_list):
        for name in genre_files:
            file = archive_file.extractfile(name)
            title, body = read_title_body(file)
            title = normalize_text(title)
            body = normalize_text(body)

            if len(title) > 0 and len(body) > 0:
                all_data.append({
                    "title": title,
                    "body": body,
                    "genre_id": i
                    })

## データ分割

データセットを70% : 15%: 15% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [8]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    title = data["title"]
    body = data["body"]
    genre_id = data["genre_id"]

    assert len(title) > 0 and len(body) > 0
    return f"{title}\t{body}\t{genre_id}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.7, 0.15, 0.15

with open(f"data/train.tsv", "w", encoding="utf-8") as f_train, \
    open(f"data/dev.tsv", "w", encoding="utf-8") as f_dev, \
    open(f"data/test.tsv", "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

7334it [00:00, 86538.89it/s]


作成されたデータを確認します。

形式: {タイトル}\t{本文}\t{ジャンルID}

In [9]:
!head -3 data/test.tsv

nttドコモ、ジョジョの奇妙な冒険25周年スマホ「jojo l-06d」を発表!荒木飛呂彦氏監修コンテンツが満載の全部入り[optimus_report]	オラララオラオラオラオラオラオラオラオラオラオラ!nttドコモは16日、今夏に発売する予定の新モデルや新しく開始するサービスなどを発表する「2012年夏モデル新商品・新サービス発表会」を開催し、人気マンガ「ジョジョの奇妙な冒険」の連載25周年を記念した限定モデル「jojo l-06d」(lgエレクトロニクス製)を発表しています。発売時期は2012年8月を予定しています。jojo l-06dは限定1万5,000台の限定モデルで、5インチサイズの大型ディスプレイを搭載したxi対応androidスマートフォン「optimus vu l-06d」をベースに、原作者・荒木飛呂彦氏が監修したコラボレーションモデルです。荒木氏は監修のほか、jojo l-06dのためだけの書き下ろしイラスト&サインが入っており、ジョジョ好きにはたまらないコンテンツが満載です。コンテンツには、荒木氏が書き下ろした壁紙を含むジョジョの人気イラストの壁紙やライブ壁紙を多数プリインストール。さらに、6種類のきせかえテーマと組み合わせることで、自分だけのお気に入りのホーム画面を設定可能になっています。また、ジョジョ第3部に登場するカーレースゲーム「f-mega」もプリインストールされており、花京院とダービー弟の名勝負を体験できます。さらに、お気に入りのスタンドと合成できるカメラアプリや、トリッシュの電卓、ウェザー・リポートウィジェット、イギーのマチキャラというように作品中に登場するキャラクターによる各種機能、「ジョジョ」の名台詞を織り交ぜたオリジナルの予測変換辞書、オリジナルデコメ絵文字、デコメテンプレートなども搭載。この他、画面サイズが4:3でほぼ文庫サイズのl-06dの端末機能を活かして、特別編集のカラー版コミック第1巻〜12巻も内蔵されています。機能的にも、optimus vu l-06dと同等で、高速データ通信規格lteによるサービス「xi(クロッシィ)」による下り最大75mbpsおよび上り最大25mbpsの高速データ通信や1.5ghzデュアルコアcpu、5インチxga(769×1024ドット)ips液晶、おサイフケータイ(felica)、ワンセ

# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [10]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import pytorch_lightning as pl

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [11]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir="/content/data",  # データセットのディレクトリ
    # model_name_or_path=PRETRAINED_MODEL_NAME,
    # tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    # learning_rate=1e-3,
    # betas=(0.9, 0.999),
    # adam_epsilon=1e-8,
    # weight_decay=0.0,
    # warmup_steps=0,
    # gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=4,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    # opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{title}\t{body}\t{genre_id}"です。

In [12]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.labels = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        source_mask = self.inputs[index]["attention_mask"].squeeze()

        label = self.labels[index].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "label": label}

    def _make_record(self, title, body, genre_id):
        # ニュース分類タスク用の入出力形式に変換する。
        input = f"{title} {body}"
        target = int(genre_id)
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 3
                assert len(line[0]) > 0
                assert len(line[1]) > 0
                assert len(line[2]) > 0

                title = line[0]
                body = line[1]
                genre_id = line[2]

                input, target = self._make_record(title, body, genre_id)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                label = torch.LongTensor([target])

                self.inputs.append(tokenized_inputs)
                self.labels.append(label)


試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [13]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                           input_max_len=512)

テストデータの1レコード目をみてみます。

In [14]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データ")
    print(data["label"])
    break

A. 入力データの元になる文字列
[CLS] ntt ドコモ 、 ジョジョ の 奇妙 な 冒険 25 周年 スマ ホ 「 jojo l - 06 d 」 を 発表! 荒木 飛 呂 彦氏 監修 コンテンツ が 満載 の 全部 入り [ optimus _ report ] オラララオラオラオラオラオラオラオラオラオラオラ! ntt ドコモ は 16 日 、 今夏 に 発売 する 予定 の 新 モデル や 新しく 開始 する サービス など を 発表 する 「 2012 年 夏 モデル 新 商品 ・ 新 サービス 発表 会 」 を 開催 し 、 人気 マンガ 「 ジョジョ の 奇妙 な 冒険 」 の 連載 25 周年 を 記念 し た 限定 モデル 「 jojo l - 06 d 」 ( lg エレクトロニクス 製 ) を 発表 し て い ます 。 発売 時期 は 2012 年 8 月 を 予定 し て い ます 。 jojo l - 06 d は 限定 1 万 5, 000 台 の 限定 モデル で 、 5 インチ サイズ の 大型 ディスプレイ を 搭載 し た xi 対応 android スマート フォン 「 optimus vu l - 06 d 」 を ベース に 、 原作 者 ・ 荒木 飛 呂 彦氏 が 監修 し た コラボレーション モデル です 。 荒木 氏 は 監修 の ほか 、 jojo l - 06 d の ため だけ の 書き下ろし イラスト & サイン が 入っ て おり 、 ジョジョ 好き に は たまらない コンテンツ が 満載 です 。 コンテンツ に は 、 荒木 氏 が 書き下ろし た 壁紙 を 含む ジョジョ の 人気 イラスト の 壁紙 や ライブ 壁紙 を 多数 プリインストール 。 さらに 、 6 種類 の きせ か え テーマ と 組み合わせる こと で 、 自分 だけ の お気に入り の ホーム 画面 を 設定 可能 に なっ て い ます 。 また 、 ジョジョ 第 3 部 に 登場 する カーレースゲーム 「 f - mega 」 も プリインストール さ れ て おり 、 花京院 と ダービー 弟 の 名 勝負 を 体験 でき ます 。 さらに 、 お気に入り の スタンド と 合成 できる カメラアプリ や 、 

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [15]:
class BertFineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        # 事前学習済みモデルの読み込み
        self.model = AutoModelForSequenceClassification.from_pretrained(hparams.model_name_or_path, num_labels=hparams.num_labels)

        if hparams.freeze_transformer:
            for param in self.model.bert.parameters():
                param.requires_grad = False

        # トークナイザーの読み込み
        self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path, do_lower_case=True, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["label"]

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels
        )

        loss = outputs[0]
        return loss

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        loss = self._step(batch)
        self.log("train_loss", loss)
        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        loss = self._step(batch)
        self.log("val_loss", loss)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        """バリデーション完了処理"""
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("val_loss", avg_loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        loss = self._step(batch)
        self.log("test_loss", loss)
        return {"test_loss": loss}

    def test_epoch_end(self, outputs):
        """テスト完了処理"""
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        self.log("test_loss", avg_loss, prog_bar=True)

    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          betas=self.hparams.betas,
                          eps=self.hparams.adam_epsilon)

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)


# 転移学習を実行

In [16]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数
    "train_batch_size":  64,
    "eval_batch_size":   8,
    "num_train_epochs":  8,
    "num_labels":        9,  # ラベルのカテゴリ数

    "model_name_or_path":     PRETRAINED_MODEL_NAME,
    "tokenizer_name_or_path": PRETRAINED_MODEL_NAME,

    "learning_rate":     1e-2,  # タスクに応じて要調整
    "betas":             (0.9, 0.999),
    "adam_epsilon":      1e-8,
    "weight_decay":      0.0,
    "warmup_steps":      30,
    "gradient_accumulation_steps": 1,

    "freeze_transformer": True,
    })
args = argparse.Namespace(**args_dict)

# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     "/content/checkpoints", 
#     monitor="val_loss", mode="min", save_top_k=1
# )

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    # amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    # checkpoint_callback=checkpoint_callback,
)

In [17]:
# Linear Probingの実行
model = BertFineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(LP_MODEL_DIR)
model.model.save_pretrained(LP_MODEL_DIR)

del model

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=8` reached.


In [18]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  512,  # 入力文の最大トークン数
    "train_batch_size":  8,
    "eval_batch_size":   8,
    "num_train_epochs":  4,
    "num_labels":        9,  # ラベルのカテゴリ数

    "model_name_or_path":     LP_MODEL_DIR,
    "tokenizer_name_or_path": LP_MODEL_DIR,

    "learning_rate":     5e-6,  # タスクに応じて要調整
    "betas":             (0.9, 0.999),
    "adam_epsilon":      1e-8,
    "weight_decay":      0.0,
    "warmup_steps":      30,
    "gradient_accumulation_steps": 1,

    "freeze_transformer": False,
    })
args = argparse.Namespace(**args_dict)

# checkpoint_callback = pl.callbacks.ModelCheckpoint(
#     "/content/checkpoints", 
#     monitor="val_loss", mode="min", save_top_k=1
# )

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    # amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    # checkpoint_callback=checkpoint_callback,
)

In [19]:
# Fine-tuningの実行
model = BertFineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
model.tokenizer.save_pretrained(MODEL_DIR)
model.model.save_pretrained(MODEL_DIR)

del model

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClass

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=4` reached.


# 学習済みモデルの読み込み

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# トークナイザー（SentencePiece）
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, do_lower_case=True, is_fast=True)

# 学習済みモデル
trained_model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# テストデータに対する予測精度を評価

In [21]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "test.tsv", 
                          input_max_len=args.max_input_length)

test_loader = DataLoader(test_dataset, batch_size=32, num_workers=4)

trained_model.eval()

outputs = []
confidences = []
targets = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['source_ids']
        input_mask = batch['source_mask']
        if USE_GPU:
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()

        outs = trained_model(input_ids=input_ids, 
            attention_mask=input_mask, 
            return_dict=True)
        
        logits = outs["logits"]
        pred_label = logits.argmax(dim=1, keepdim=True)
        conf = logits.softmax(dim=1).gather(dim=1, index=pred_label).squeeze().cpu().numpy().tolist()
        
        pred_label = pred_label.squeeze().cpu().numpy().tolist()
       
        target = batch["label"].tolist()

        outputs.extend(pred_label)
        confidences.extend(conf)
        targets.extend(target)
        

  0%|          | 0/35 [00:00<?, ?it/s]

## accuracy

In [22]:
metrics.accuracy_score(targets, outputs)

0.9418181818181818

## ラベル別精度

[accuracy, precision, recall, f1-scoreの意味](http://ibisforest.org/index.php?F値)

In [23]:
print(metrics.classification_report(targets, outputs))

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       130
           1       0.95      0.94      0.95       121
           2       0.93      0.89      0.91       123
           3       0.92      0.83      0.87        82
           4       0.93      0.96      0.94       129
           5       0.92      0.96      0.94       141
           6       0.97      0.98      0.98       127
           7       0.98      0.97      0.98       127
           8       0.94      0.96      0.95       120

    accuracy                           0.94      1100
   macro avg       0.94      0.94      0.94      1100
weighted avg       0.94      0.94      0.94      1100



## 確信度の上下限

In [24]:
min(confidences), max(confidences)

(0.3949939012527466, 1.0)